In [ ]:
#Penglab (Abuse of Google Colab for fun and profit)
#by mxrch

#Choose what you want to install
hashcat = True
john = False
hydra = False

#Which shell ? (only one, or none)
#⚠️ Don't select a shell to run bash commands in the Google Colab way (you'll see). Stable.
ssh = True #It uses ngrok. Stable and interactive, but take a little time to setup
python_shell = False #Useful for little hashes, but not for long hours, since you'll see the output only when it finish. Not interactive, but stable.
bash_shell = False #Useful for big hashes. but you'll don't see your input (surely a Google Colab protection). Stable.

#Wordlists (see weakpass.com to download them)
wordlists_dir = "wordlists"

rockyou = True #For CTFs, especially Hack The Box (133.44 Mb)
hashesorg2019 = False #Very heavy (12.79 Gb) but has a good rate, if you're determinated

#---------------------------

if (python_shell and bash_shell) or (bash_shell and ssh) or (ssh and python_shell) :
    print("Please do a choice")
    exit()

In [ ]:
#Prevent random disconnects
#This cell runs JS code to automatic reconnect to runtime.
import IPython
from google.colab import output

display(IPython.display.Javascript('''
 function ClickConnect(){
   btn = document.querySelector("colab-connect-button")
   if (btn != null){
     console.log("Click colab-connect-button"); 
     btn.click() 
     }
   
   btn = document.getElementById('ok')
   if (btn != null){
     console.log("Click reconnect"); 
     btn.click() 
     }
  }
  
setInterval(ClickConnect,60000)
'''))

print("Done.")

<IPython.core.display.Javascript object>

Done.


In [ ]:
#Wordlists
import os

os.system("wordlists_dir={}".format(wordlists_dir))
!mkdir ./$wordlists_dir
if rockyou:
    !printf "[+] Downloading the Rockyou wordlist...\n"
    !cd $wordlists_dir && wget https://download.weakpass.com/wordlists/90/rockyou.txt.gz
    !printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
    !cd $wordlists_dir && gunzip rockyou.txt.gz
    !printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep rockyou)"

if hashesorg2019:
    !printf "[+] Downloading the HashesOrg2019 wordlist...\n"
    !cd $wordlists_dir && wget https://download.weakpass.com/wordlists/1851/hashesorg2019.gz
    !printf "[+] Wordlist downloaded !\n[+] Extraction...\n"
    !cd $wordlists_dir && gunzip hashesorg2019.gz
    !printf "[+] Finished !\n[+] Location : $(pwd)/$wordlists_dir/$(ls wordlists | grep hashesorg2019)"

In [ ]:
#Install of hashcat
if hashcat:
    print("Installation of hashcat...")
    !apt install cmake build-essential -y
    !apt install checkinstall git -y
    !git clone https://github.com/hashcat/hashcat.git && cd hashcat && make -j 8 && make install

In [ ]:
#Install of john
if john:
    print("Installation of john...")
    !apt-get install john -y

In [ ]:
#Install of hydra
if hydra:
    print("Installation of hydra...")
    !apt install cmake build-essential -y
    !apt install checkinstall git -y
    !git clone https://github.com/vanhauser-thc/thc-hydra.git && cd thc-hydra && ./configure && make && make install

In [ ]:
#Setting up shells
from termcolor import colored
import os

if ssh:
    print("Setting up SSH...")
    !pip install git+https://github.com/demotomohiro/remocolab.git
    import remocolab
    remocolab.setupSSHD(tunnel = "argotunnel")

elif python_shell:
    os.system("touch /tmp/cmd && touch /tmp/status")

    template_cmd = "echo -n $(whoami)[$(hostname)[$(pwd) &> /tmp/status"
    os.system("bash -c '{}'".format(template_cmd))
    output = {"cmd": "", "status": ""}
    with open('/tmp/cmd', 'r') as f:
        output["cmd"] = f.read()
    with open('/tmp/status', 'r') as f:
        output["status"] = f.read()
    prefixes = output["status"].split("[")
    path = prefixes[2].replace('\n', '')
    prefix = colored(prefixes[0] + "@" + prefixes[1], "red") + ":" + colored(path, "cyan") + "$ "
    print("")

    while 1:
        print(prefix, end='')
        cmd = input()
        template_cmd = "cd {} && ".format(path) + "" + cmd + " &> /tmp/cmd ; echo $(whoami)[$(hostname)[$(pwd) &> /tmp/status"
        os.system("bash -c '{}'".format(template_cmd))
        output = {"cmd": "", "status": ""}
        with open('/tmp/cmd', 'r') as f:
            output["cmd"] = f.read()
        with open('/tmp/status', 'r') as f:
            output["status"] = f.read()
        prefixes = output["status"].split("[")
        path = prefixes[2].replace('\n', '')
        prefix = colored(prefixes[0] + "@" + prefixes[1], "red") + ":" + colored(path, "cyan") + "$ "
        print(output["cmd"])

elif bash_shell:
    !/bin/bash

else:
    print('\n🚨🚨🚨\n🎉 Your environment is ready !\nWant to run it from the code blocks ?')
    print('Just use the code block below and start your commands by a "!" (make sure to uncomment them).')
    print('▶️ When you\'re ready, hit the run button at the left of the code block !\n')
    print('|\n|\n|\nV')

In [ ]:
#Put your code here :
# !john --help
!nvidia-smi #check GPU
!hashcat -m0 -b

Sun Mar  6 23:45:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------